# Geocode Victoria's exposure sites
## Uses the awesome geocoding engine from https://opencagedata.com/
No guarantees given whatsoever about accuracy of the results. Use at your own risk.

In [ ]:
!pip install opencage pandas fastparquet

In [ ]:
import pandas as pd
from time import sleep

In [ ]:
api_key = "" # insert your API key here.

In [ ]:
dataset_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSNouXrJ8UQ-tn6bAxzrOdLINuoOtn01fSjooql0O3XQlj4_ldFiglzOmDm--t2jy1k-ABK6LMzPScs/pub?gid=1075463302&single=true&output=csv"

In [ ]:
exposure_sites = pd.read_csv(dataset_url)

In [ ]:
from opencage.geocoder import OpenCageGeocode

geocoder = OpenCageGeocode(api_key)
geocoded = []
for site in exposure_sites.itertuples():
    address = f"{site.Site_streetaddress.strip()}, {site.Suburb.strip()}, {site.Site_state.strip()}, {site.Site_postcode}"
    results = geocoder.geocode(query=address,
                               countrycode="AU",
                               limit=1,
                               bounds=[140.32471,-39.24360,150.27832,-33.59022], # uses rough bounding box for Victoria
                               no_annotations=1)
    sleep(1)
    geocoded = geocoded + [pd.DataFrame.from_dict(pd.json_normalize(results)).assign(Index = site.Index)]
    
geocoded_df = pd.concat(geocoded, sort=True).set_index("Index")

In [ ]:
geocoded_exposure_sites = exposure_sites.merge(geocoded_df, left_index=True, right_index=True)
geocoded_exposure_sites.to_parquet("geocoded_exposure_sites.parquet")
geocoded_exposure_sites.to_csv("geocoded_exposure_sites.csv")

### The results do not always geocode to a point. Sometimes the engine will return a neighbourhood!

In [ ]:
geocoded_exposure_sites["components._type"].value_counts()

### The engine gives an estimate of the match confidence. 10 is good.

In [ ]:
geocoded_exposure_sites.confidence.value_counts()